<a href="https://colab.research.google.com/github/kingsgameBen/MLbasic_0223/blob/main/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0	T-shirt/top 短袖上衣 \
1	Trouser 褲子 \
2	Pullover 套頭毛衣/套衫 \
3	Dress 洋裝 \
4	Coat 外套 \
5	Sandal 涼鞋 \
6	Shirt 襯衫 \
7	Sneaker 帆布鞋、運動鞋 \
8	Bag 包包\
9	Ankle boot 短靴

In [ ]:
from tensorflow.keras.datasets.fashion_mnist import load_data
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
trans = [
	"T-shirt",
	"Trouser",
	"Pullover",
	"Dress",
	"Coat",
	"Sandal",
	"Shirt",
	"Sneaker",
	"Bag",
	"Ankle boot"
]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[1]*x_train.shape[2])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_train[215])

In [ ]:
import pandas as pd
pd.DataFrame(x_train[215])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import os
from tensorflow.keras.losses import CategoricalCrossentropy
if os.path.exists("mlp.h5"):
    print("模型存在, 直接讀取")
    model = load_model("mlp.h5")
else:
    total_train_size = x_train.shape[1]*x_train.shape[2]
    layers = [
            # 64, 128, 256, 512..
            #   784 * 128 + 128(bias)
            # 測試最終採用256 => 784 * 256 + 256(bias)
            Dense(256, activation="relu", input_dim=total_train_size),
            Dropout(0.25),
            #   128 * 10 + 10(bias)
            Dense(10, activation="softmax")
    ]
    model = Sequential(layers)
    model.compile(loss=CategoricalCrossentropy(),
              metrics=["accuracy"],
              optimizer="adam")
model.summary()

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print(y_train[0])
print(y_train_cat[0])

In [ ]:
x_train_shape = x_train.reshape(-1, 784) / 255.0
x_test_shape = x_test.reshape(-1, 784)  / 255.0

存檔點(ModelCheckpoint)[https://keras.io/zh/callbacks/#modelcheckpoint ]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
             EarlyStopping(patience=5, restore_best_weights=True),
             ModelCheckpoint("mlp.h5", save_best_only=True)
]
model.fit(x_train_shape,
          y_train_cat,
          batch_size=200,
          epochs=50,
          validation_split=0.1,
          callbacks=callbacks,
          verbose=2)

In [ ]:
model.evaluate(x_test_shape, y_test_cat)
# hdf5 層級資料格式（Hierarchical Data Format：HDF）
# model.save("mlp.h5"), 但是要在fit階段就讀取上次儲存的模型,所以儲存寫在fit的callbacks裡:ModelCheckpoint

In [ ]:
from sklearn.metrics import confusion_matrix
pre = model.predict_classes(x_test_shape)
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             columns=[ str(i) + "(預測)" for i in range(10)],
             index=[ str(i) + "(正確)" for i in range(10)])

In [ ]:
import numpy as np
idx = np.nonzero(pre != y_test)[0]
print(idx)
print(len(idx))
# 取段操作 顯示200個就好
idx = np.nonzero(pre != y_test)[0][:200]
false_img = x_test[idx]
false_pre = pre[idx]
false_label = y_test[idx]
plt.figure(figsize=(20, 60))
width = 10
height = len(idx) // width + 1
for i in range(len(idx)):
    plt.subplot(height, width, i+1)
    plt.imshow(false_img[i])
    t = "[Ori]:{}\n[Pre]:{}".format(trans[false_label[i]], trans[false_pre[i]])
    plt.title(t)
    plt.axis("off")

In [ ]:
from PIL import Image
import requests
headers ={
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"
}
url = input("url: ")

response = requests.get(url, headers=headers, stream=True, verify=False)
img = Image.open(response.raw).resize((28, 28)).convert("L")
# 搜尋的圖片，多為白色底背景，故以255-背景色數值，達到數值反轉，才能跟原本訓練資料一樣的背景
img_np = 255 - np.array(img)
img_np_r = img_np.reshape(1, 784) / 255.0
proba = model.predict(img_np_r)[0]
for i in range(10):
    print(trans[i], "的機率:", proba[i])
ans = model.predict_classes(img_np_r)[0]
print("答案:", trans[ans])
plt.imshow(img_np, "gray")